# 深層学習(Deep Neural Networks)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kyorin-phys/MLIntro2025/blob/main/4/4.ipynb)

* 入力データ
* 出力データ（正解）

入力データと出力データの組を**学習**する

入力データから出力データを再現できるような**関数**を作ることができるものを**機械学習**という。

関数はパラメータを持つ。線形回帰なら、パラメータ（傾きと切片）を調節して、予測データと正解データの「誤差」が
最小となるようなパラメータを求める。(「誤差」の評価はスケール変数とカテゴリ変数で異なる。)

深層ニューラルネットワークは隠れ層を入れることでパラメータの数を増やして、近似的な**関数**を作る仕組みである。

生物のニューロンにおいて、刺激の総和が閾値を超えると発火するという概念をまねたものを人工ニューロン（パーセプトロン）という。

単層パーセプトロン(SLP)：入力と出力しかなく、入力の重み付き和が閾値をこえるかどうかが出力である。線形分類（直線や平面でデータが完全に分けられる）しかできない。
$$
f(x_1, x_2) = w_1 x_1 + w_2 x_2 + b > 0 
$$
<img src='SLP.png' width='10%'>


多層パーセプトロン(MLP)：隠れ層（中間層）を入れたもの。層を増やすことでパラメータを増やすことと、
非線形な活性化関数（例：シグモイド関数、階段関数、ReLU、softmax）を使うことで非線形な分類ができるようになる。

$$
y = g(x),\,  z = g(y)
$$
<img src='MLP.png' width='20%'>

ニューラルネットワーク(NN)はより一般的な計算モデルで、MLPもその1つ。深層学習はDeep Neural Networkで複数の中間層を意味する。

[様々なNN](https://www.asimovinstitute.org/wp-content/uploads/2019/04/NeuralNetworkZo19High.png)

## 誤差逆伝播法（バックプロパゲーション）
重みパラメータ$w$とバイアス$b$を調整して、入力から求めた予測値と正解データとの誤差が小さくするよう最適化して$(w,b)$を決定する。
よいモデルであれば、どんな入力に対しても正解データに近い予測が得られるはず。


In [ ]:
# @title {display-mode: "form"}
# 活性化関数
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def step(z):
    return np.heaviside(z, 0.5)
def ReLU(z):
    return np.maximum(0, x)

x = np.linspace(-10, 10, 200)
y1 = sigmoid(x)
y2 = step(x)
y3 = ReLU(x)
plt.figure(figsize=(4,2))
plt.plot(x, y1, label='sigmoid')
plt.plot(x, y2, label='step')
plt.plot(x, y3, label='ReLU')
plt.legend()
plt.ylim(0,1.1)
plt.xlabel('x')
plt.ylabel('y')
#plt.savefig('activation.png')
plt.show()

In [ ]:
# アヤメデータの読み込み
import numpy as np
from sklearn.datasets import load_iris

iris = load_iris()
print(iris.data[:2])
print(iris.data.shape)
print(iris.target)

In [ ]:
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
# データの正規化(平均0、分散1)
scaler = preprocessing.StandardScaler()
scaler.fit(iris.data)
#x = iris.data
x = scaler.transform(iris.data)
print(x[:2])
print(x.mean(), x.std())

In [ ]:
# one hot 表現　0 -> [1,0,0], 1 -> [0,1,0], 2 -> [0,0,1] 
t = to_categorical(iris.target) # カテゴリー変数をone-hot表現に変換
print(t[[49, 99, 149]])

<img src="split.png" width="30%">

In [ ]:
# 訓練用と検証用に分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, t, test_size=0.3, random_state=0) 
#x_train, x_test, y_train, y_test = train_test_split(x, t, test_size=0.3, random_state=0, stratify=t) # データセットの比率に合わせて分割（層化サンプリング）
#stratify=t で各カテゴリの比率が同程度になるように分割

## ニューラルネットワーク

* 入力　4次元ベクトル（アヤメの特徴量）
* 出力　3次元ベクトル（アヤメの種類の確率）
* 中間層　M次元ベクトル（N層）M,Nの値は任意（ハイパーパラメータ）

## 使用するライブラリ
* scikit-learn 訓練用と検証用の分割、前処理などに使う
* tensorflow 直接表には出てこないが、多次元配列の計算がGPUで高速化される
* keras ニューラルネットワークを扱う
* (pytorch というライブラリもあるが、ここでは扱わない)

`model = Sequential([])`

の中に層の種類と大きさ、活性化関数の種類を指定する。
* `Input()` 入力層
* `Dense()` 全結合層

2次元画像に有用なCNNなどの発展形では更に増える

誤差関数 loss の取り方

* 回帰問題も扱えるがここでは略
* 2値分類　`BinaryCrossentropy()` 1(True), 0(False) 
* 多クラス分類 (one-hot エンコーディング) `CategoricalCrossentropy()` 1=[1,0,0],2=[0,1,0],3=[0,0,1]
* 多クラス分類（整数ラベル） `SparseCategoricalCrossentropy()` 1,2,3

この例ではアヤメのクラスが3種類あり、one-hotエンコーディングしているので`CategoricalCrossentropy()`を使う。

犬か猫かという2値分類では`BinaryCrossentropy()`を使う。  

<img src='dnn.png' width=50%>

In [ ]:
# この例では中間層はN=2層, どちらも大きさはM=6
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from keras.losses import CategoricalCrossentropy
model = Sequential([
    Input(shape=(4,)), # 入力層　（x_trainの形状に対応させる）
    Dense(6, activation='relu'), # 中間層
    Dense(6, activation='relu'), # 中間層
    Dense(3, activation='softmax') # 出力層 (y_trainの形状)
])

model.compile(optimizer='adam', loss=CategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

In [ ]:
# 訓練 epochs, batch_size はハイパーパラメータ
# 1epochあたり 全サンプル数/バッチサイズ の回数だけ重みが更新される
#tf.random.set_seed(0)
history = model.fit(x_train, y_train, epochs=200, batch_size=32, validation_data=(x_test, y_test)) 

In [ ]:
# 学習の様子を確認する　loss:誤差できるだけ小さく、accuracy:正解率できるだけ大きくなるのがベター
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['val_loss'], label='val_loss')
plt.plot(history.history['loss'], label='loss')
plt.ylim(0, 1)
plt.legend()

In [ ]:
# 訓練後の評価
loss, accuracy = model.evaluate(x_test, y_test)
print(loss, accuracy)

In [ ]:
# 予測値
y_pred = model.predict(x_test)
print(y_pred[:10])
print(y_test[:10])
maxindex = np.argmax(y_pred, axis=1) # axis=0は行方向、axis=1は列方向で最大値のインデックス
print(maxindex[:20])
t_test = np.argmax(y_test, axis=1)
print(t_test[:20])

In [ ]:
# 混同行列
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(t_test, maxindex)
print(cm)

In [ ]:
# 混同行列の可視化
import seaborn as sns
import matplotlib.pyplot as plt

# Create a heatmap of the confusion matrix
#plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', cbar=False)

# Add labels and title
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.title('Confusion Matrix Heatmap (Multi-class)')
plt.xticks(ticks=[0.5, 1.5, 2.5], labels=['Setosa', 'Versicolor', 'Virginica'])
plt.yticks(ticks=[0.5, 1.5, 2.5], labels=['Setosa', 'Versicolor', 'Virginica'], rotation=0)

# Display the plot
plt.show()

In [ ]:
# 前回の練習問題にあったpenguinデータセットで同様の分類ができる。
# サイズは大きいが乳がんデータ(sklear.datasets.load_breast_cancer)は2値分類であるので、損失関数はCategoricalCrossentropy からBinaryCrossentropyに、softmaxをsigmoidに変えればよい